In [11]:
#hide
# default_exp utilities
from nbdev.showdoc import *

# Utilities

> Scripts and functions used by other modules

In [12]:
#export
import glob
import os
import subprocess
import tempfile

import git
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
from fastcore.script import *

# Project metadata/configuration

Metadata and configuration for the project are stored in the `.kicad_helpers_config` directory (including [kibot](https://github.com/INTI-CMNB/KiBot) config files for producting manufacturing outputs via continuous integration). Additional metadata (e.g., project summary, website, and manufacturing details) is stored in the project's `kitspace.yaml` file which makes it easy for other people to manufacture boards using https://kitspace.org/.

In [15]:
#export
def get_git_root(path="."):
    # Find the current projects' root directory
    git_repo = git.Repo(path, search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")

In [16]:
#export
@call_parse
def setup_test_repo(root:Param("project root directory", str)="_temp"):
    """Setup a test KiCad repository to test against.
    """
    if not os.path.exists(root):
        subprocess.check_call(f"git clone https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad { root }", shell=True)

In [17]:
#hide
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

In [19]:
#export
def _set_root(root):
    """If `root` is the default value ("."), use the project's git root
    or override with the environment variable `KH_PROJECT_ROOT` if it
    exists.
    """
    if root == ".":
        # Use defaults
        root = get_git_root(".")
        
        # Override with environment variable if set
        root = os.getenv("KH_PROJECT_ROOT", root)
    return root

In [20]:
#export
def get_project_name(root="."):
    """Get the project name based on the name of the KiCad `*.pro` file.
    """
    root = _set_root(root)
    return os.path.splitext(os.path.split(glob.glob(os.path.join(root, "*.pro"))[0])[1])[0]


In [22]:
#hide
print(f"get_project_name() = { get_project_name(root) }")
assert get_project_name(root) == "40-channel-hv-switching-board"

get_project_name() = 40-channel-hv-switching-board


In [24]:
#export
def get_project_metadata(root="."):
    """Get the project metatdata from the `kitspace.yaml` file.
    """
    root = _set_root(root)

    # Default metadata if there's no existing `kitspace.yaml` file.
    metadata = {"summary": "A description for your project",
                "site": "https://example.com # A site you would like to link to (include http:// or https://)",
                "color": "black"
    }
    
    try:
        # If there's an existing `kicad.yaml` file, those settings override the defaults.
        with open(os.path.join(root, "kitspace.yaml")) as f:
            metadata.update({k: v for k, v in load(f, Loader=Loader).items() if k in ["summary", "site", "color"]})
    except FileNotFoundError:
        pass

    # Add the project name
    metadata["project_name"]=get_project_name(root)
    return metadata


In [25]:
#hide
print(f"get_project_metadata() = { get_project_metadata(root) }")
assert get_project_metadata(root) == {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}

get_project_metadata() = {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}


In [28]:
#export
def get_schematic_path(root="."):
    """Get the path to the KiCad schematic.
    """
    root = _set_root(root)
    return os.path.join(root, get_project_name(root) + ".sch")

In [29]:
#hide
print(f"get_schematic_path() = { get_schematic_path(root) }")
assert os.path.exists(get_schematic_path(root))

get_schematic_path() = /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/40-channel-hv-switching-board.sch


In [30]:
#export
def get_bom_path(root="."):
    """Get the path to the BOM.
    """
    root = _set_root(root)
    return os.path.join(root, "manufacturing", "default", get_project_name(root) + "-BOM.csv")

In [32]:
#hide
print(f"get_bom_path() = { get_bom_path(root) }")
assert os.path.exists(get_bom_path(root))

get_bom_path() = /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/manufacturing/default/40-channel-hv-switching-board-BOM.csv


In [37]:
#export
def get_gitignore_list(root="."):
    root =_set_root(root)
    with open(f"{ root }/.gitignore") as f:
        gitignore = f.readlines()
    return "|".join([line.strip() for line in gitignore])

In [42]:
#hide
print(f"get_gitignore_list() = { get_gitignore_list(root) }")
assert get_gitignore_list(root) == "_autosave*|*bak|*cache.*|*rescue.lib|*.xml|.ipynb_checkpoints"

get_gitignore_list() = _autosave*|*bak|*cache.*|*rescue.lib|*.xml|.ipynb_checkpoints


In [43]:
#export
def in_gitignore(filename):
    try:
        if len(subprocess.check_output(f"echo '{ filename }' | git check-ignore --stdin --no-index", shell=True)):
            return True
    except subprocess.CalledProcessError:
        pass
    return False

In [48]:
#hide
assert in_gitignore("project.sch-bak") == True

In [49]:
#export
def install_python_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])